
# Λ<sub>c</sub><sup>+</sup> Reconstruction using ROOT TMVA

As described in the [introduction](Introduction.md), it is possible to use the TMVA (Toolkit for Multivariate Data Analysis) with ROOT to perform multivariate analysis on high energy physics data.
Here is shown how to use 3 of the TMVA methods:

* Linear cuts 
* Neural Networks of type Multilayer Perceptron (MLP)
* Boosted Decision Tree (BDT)

The results of this analysis are presented in [result file](introduction.md).<br>
With the latest versions of ROOT it is possible to use all functions in python codes. This feature is called pyROOT, an example of its use is shown here.

<p>&nbsp;</p>

In [1]:
from ROOT import TMVA, TCut, TFile, gSystem  #import ROOT necessary functions
from math import floor                       #used to evaluate the minimum between two values
import sys                                   #used in case of errors 

Welcome to JupyROOT 6.22/03


## 1. Declaration of Factory

All the details on using the following functions can be found in in the [TMVA Users Guide](References/TMVAUsersGuide.pdf).<br>
Using the TFile class of ROOT, the [TMVA_output_4_6.root](TMVA_output_4_6.root) output file is opened. <br>
Then the TMVA factory object is declared. The options used in the declaration are described in the table:

Option          |Given Value     |Description
----------------|----------|---------------------------
V|False|Verbose flag
Color|True|Flag for colored output
DrawProgressBar|True|Draw progress bar to display training, testing and evaluation schedule 
Transformations||List of transformations to test. For example with "I;D;P;U;G" string identity, decorrelation, PCA, uniform and Gaussian transformations will be applied
AnalysisType|Classification|Set the analysis type

<br>


In [2]:
outputFile = TFile( "TMVA_output_4_6.root", 'RECREATE' )
TMVA.Tools.Instance()
factory = TMVA.Factory( "TMVAClassification", outputFile,
                       "!V:Color:DrawProgressBar:Transformations=I;D;P;G,D:AnalysisType=Classification" )

## 2. DataLoader Declaration, adding variables and setting the data set

The <b>AnalysisResults.root</b> file is defined as input (more details on its content are given in the [analysis results](README.md)). Once the Dataloader has been declared, the variables contained in the root trees of the input file are added. the AddVariable function used requires the following arguments:

* A string containing the name of the variable (the same as the root tree)
* A label for variable (if not present the variable name is used)
* data point type (second argument if the label for variable is missing)

It is also possible to define a combination of variables or an operation with variable using the syntax ":=" in the first argument. An example of this is represented by the <i>CtK0S</i> variable.<br><br>
Spectator Variables have also been added. These are part of the data set but are not used for multivariate analysis in the Training or Test phases. They can for example be used for correlation tests.<br><br>
More details on the variables used and their choice can be found in [analysis results](README.md).

In [3]:
#input file check 
if gSystem.AccessPathName( "AnalysisResults.root" ) != 0: 
    sys.exit("Input file not found")

#open input file
input= TFile.Open("AnalysisResults.root")

#declaration
dataloader = TMVA.DataLoader("dataset")

#Add Variable
dataloader.AddVariable( "massK0S", 'F' )
dataloader.AddVariable( "CosThetaStar", 'F' )
dataloader.AddVariable( "combinedProtonProb", 'F')
dataloader.AddVariable( "signd0", 'F')
dataloader.AddVariable( "tImpParV0", 'F')
dataloader.AddVariable( "tImpParBach", 'F' )
dataloader.AddVariable( "CtK0S := DecayLengthK0S*0.497/v0P", 'F')
dataloader.AddVariable( "cosPAK0S", 'F' )

#Add spectator variables
dataloader.AddSpectator( "nSigmaTOFpr", 'F' )
dataloader.AddSpectator( "nSigmaTPCpr", 'F' )
dataloader.AddSpectator( "nSigmaTPCpi", 'F' )
dataloader.AddSpectator( "nSigmaTPCka", 'F' )
dataloader.AddSpectator( "bachelorEta", 'F' )
dataloader.AddSpectator( "LcPt", 'F' )
dataloader.AddSpectator( "bachelorPt", 'F')
dataloader.AddSpectator( "massLc2K0Sp", 'F' )
dataloader.AddSpectator( "massLc2Lambdapi",  'F' )
dataloader.AddSpectator( "massLambda", 'F' )
dataloader.AddSpectator( "massLambdaBar", 'F' )
dataloader.AddSpectator( "V0positivePt", 'F' )
dataloader.AddSpectator( "V0negativePt", 'F' )
dataloader.AddSpectator( "dcaV0pos", 'F' )
dataloader.AddSpectator( "dcaV0neg", 'F' )
dataloader.AddSpectator( "v0Pt", 'F' )
dataloader.AddSpectator( "dcaV0", 'F' )
dataloader.AddSpectator( "V0positiveEta",'F' )

Warning in <TClass::Init>: no dictionary for class AliRDHFCutsLctoV0 is available
Warning in <TClass::Init>: no dictionary for class AliRDHFCuts is available
Warning in <TClass::Init>: no dictionary for class AliAnalysisCuts is available
Warning in <TClass::Init>: no dictionary for class AliVCuts is available
Warning in <TClass::Init>: no dictionary for class AliESDtrackCuts is available
Warning in <TClass::Init>: no dictionary for class AliAODPidHF is available
Warning in <TClass::Init>: no dictionary for class AliAnalysisUtils is available
Warning in <TClass::Init>: no dictionary for class AliEventCuts is available
Warning in <TClass::Init>: no dictionary for class AliNormalizationCounter is available
Warning in <TClass::Init>: no dictionary for class AliCounterCollection is available


## 3. Import signal and background trees

In [4]:
# signal and backg trees for training and testing 
signal      = input.Get( "treeList_2_25_2_25_Sgn")
background  = input.Get( "treeList_2_25_2_25_Bkg")

<br>
We pass to the Training and Test the same trees for signal and background, this will then be divided into two subunits that will be used separately for training and testing. The <b>AddSignalTree</b> and <b>AddBackgroundTree</b> functions need a weight (for this analysis it was decided to use unit weights).
<br>
<br>

In [5]:
signalWeight     = 1.0
backgroundWeight = 1.0

dataloader.AddSignalTree(signal, signalWeight)
dataloader.AddBackgroundTree(background, backgroundWeight)

DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree treeList_2_25_2_25_Sgn of type Signal with 1762028 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree treeList_2_25_2_25_Bkg of type Background with 1473789 events


## 4. Cuts on input data and definition of training and testing dataset 

We define TCut objects in order to select only a fraction of the input data. It was decided to use the transverse momentum of Λ<sub>c</sub><sup>+</sup>(LcPt) as discriminating variable. In the example shown, the selected range is [4-6] (GeV/c). More details on the number and type of events in this range are illustrated in the file [analysis results](README.md).
<br>
<br>

In [6]:
mycuts = TCut("LcPt<6 && LcPt>4")
mycutb = TCut("LcPt<6 && LcPt>4")

<br>
After the cut on the transverse momentum, the number of events to be used in the training and in the testing is decided as follows:

* Training and testing of the Signal is done with the minimum value between half of the events remaining after the cut and 100000

* Bakground training and testing is done with the minimum value between half of the events remaining after the cut and 200000

Training and testing are obviously done with different events randomly selected within the trees.
<br>
<br>


In [7]:
nTrainingEventsSgn = floor(min(signal.GetEntries("LcPt<6 && LcPt>4")*0.5,100000))
nTrainingEventsBkg = floor(min(background.GetEntries("LcPt<6 && LcPt>4")*0.5,200000))
nTestingEventsSgn = floor(min(signal.GetEntries("LcPt<6 && LcPt>4")*0.5,100000))
nTestingEventsBkg = floor(min(background.GetEntries("LcPt<6 && LcPt>4")*0.5,200000))

### 4.1 Preparation of Training and Test Treee
We use the <b>dataLoader.PrepareTrainingAndTestTree</b> function to apply the cuts to the input events and to prepare the trees that will be used. Option passed are:<br>

Option|Given Value|Description
------|-----------|-----------
nTrain_Signal|nTrainingEventsSgn|Number of training events of class Signal
nTest_Signal|nTestingEventsSgn|Number of test events of class Signal
nTrain_Background|nTrainingEventsBkg|Number of training events of class Background
nTest_Background|nTestingEventsBkg|Number of test events of class Background
SplitMode|Random|Method of picking training and testing events (can be Random, Alternate, Block)
NormMode|NumEvents|Overall renormalisation of event-by-event weights used in the training (NumEvents: average weight of 1 per event, independently for signal and background; EqualNumEvents: average weight of 1 per event for signal, and sum of weights for background equal to sum of weights for signal)
V|False|Verbosity (Connected to the detail on the  std_output produced) 

<br>
<br>

In [8]:
dataloader.PrepareTrainingAndTestTree( mycuts, mycutb,"nTrain_Signal=nTrainingEventsSgn:nTest_Signal=nTestingEventsSgn:nTrain_Background=nTrainingEventsBkg:nTest_Background=nTestingEventsBkg:SplitMode=Random:NormMode=NumEvents:!V")

## 5. Booking methods
To Book methods we use the <b>factory.BookMethod</b> function which needs the following arguments:

Option|Given Value|Description
------|-----------|-----------
DataLoader|dataloader|Pointer to DataLoader object
Method|TMVA.Types.kCuts|Selected method number, method numbers defined in TMVA.Types (kVariable kCuts , kLikelihood , kFisher , kKNN , kCFMlpANN , kMLP , kBDT , kDT , kRuleFit , kSVM , kMLP , kDNN , ...)
MethodTitle|Cuts|Label for method
""|""|Other named arguments which are the options for selected method.

### 5.1 Booking linear Cuts 
As described in the [introduction](Introduction.md) the first TMVA method we want to test is the simple linear cuts. In This case fourth argument of the <b>factory.BookMethod</b> function has been set as:

Option|Given Value|Description
------|-----------|-----------
H|False|Print method-specific help message
V|False|Verbose output
FitMethod|MC|Minimization method used to make the fit
EffMethod|EffSel|Selection method used to calculate the efficiency of a cut on a variable
VarProp|FSmart|Categorization of cuts

In [9]:
factory.BookMethod( dataloader, TMVA.Types.kCuts, "Cuts","!H:!V:FitMethod=MC:EffMethod=EffSel:VarProp=FSmart")

Factory                  : Booking method: Cuts
                         : 
                         : Use optimization method: "Monte Carlo"
                         : Use efficiency computation method: "Event Selection"
                         : Use "FSmart" cuts for variable: 'massK0S'
                         : Use "FSmart" cuts for variable: 'CosThetaStar'
                         : Use "FSmart" cuts for variable: 'combinedProtonProb'
                         : Use "FSmart" cuts for variable: 'signd0'
                         : Use "FSmart" cuts for variable: 'tImpParV0'
                         : Use "FSmart" cuts for variable: 'tImpParBach'
                         : Use "FSmart" cuts for variable: 'CtK0S'
                         : Use "FSmart" cuts for variable: 'cosPAK0S'


### 5.2 Booking Boosted Decision Tree 
As described in the [introduction](Introduction.md) the second TMVA method we want to test is the Boosted Decision Tree. In This case fourth argument of the <b>factory.BookMethod</b> function has been set as:

Option|Given Value|Description
------|-----------|-----------
H|False|Print method-specific help message
V|False|Verbose output
NTrees|850|Number of trees in the forest
MinNodeSize|2.5%|Minimum percentage of training events required in a leaf node (default: Classification: 5%, Regression: 0.2%)
MaxDepth|3|Max depth of the decision tree allowed
BoostType|AdaBoost|Boosting type for the trees in the forest (AdaBoost, RealAdaBoost, Bagging, AdaBoostR2, Grad)
AdaBoostBeta|0.5|Learning rate for AdaBoost algorithm
UseBaggedBoost|True|Use only a random subsample of all events for growing the trees in each iteration
BaggedSampleFraction|0.5|Relative size of bagged event sample to original size of the data sample (used whenever bagging is used (i.e. UseBaggedGrad, Bagging,)
SeparationType|GiniIndex|Separation criterion for node splitting (CrossEntropy, GiniIndex, GiniIndexWithLaplace, MisClassificationError, SDivSqrtSPlusB, RegressionVariance)
nCuts|20|Number of grid points in variable range used in finding optimal cut in node splitting

In [10]:
factory.BookMethod( dataloader, TMVA.Types.kBDT, "BDT","!H:!V:NTrees=850:MinNodeSize=2.5%:MaxDepth=3:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" )

### 5.2 Booking Neural Networks of type Multilayer Perceptron
As described in the [introduction](Introduction.md) the second TMVA method we want to test is the Boosted Decision Tree. In This case fourth argument of the <b>factory.BookMethod</b> function has been set as:

Option|Given Value|Description
------|-----------|-----------
H|False|Print method-specific help message
V|False|Verbose output
NeuronType|tanh|Neuron activation function type
VarTransform|N|List of variable transformations performed before training, e.g., D_Background,P_Signal,G,N_AllClasses for: Decorrelation, PCA-transformation, Gaussianisation, Normalisation, each for the given class of events ('AllClasses' denotes all events of all classes, if no class indication is given, 'All' is assumed)
NCycles|600|Number of training cycles
HiddenLayers|N+5|Specification of hidden layer architecture
TestRate|5|Test for overtraining performed at each #th epochs
UseRegulator|False|Use regulator to avoid over-training

In [11]:
factory.BookMethod(dataloader, TMVA.Types.kMLP, "MLP","H:!V:NeuronType=tanh:VarTransform=N:NCycles=600:HiddenLayers=N+5:TestRate=5:!UseRegulator" )

## 6. Train, Test and Evaluate all Methods

After booking all the methods to be analyzed, we use the functions: <b>factory.TrainAllMethods()</b>, <b>factory.TestAllMethods()</b> and <b>factory.EvaluateAllMethods()</b> to perform Training Test and Evaluation of the results of the booking methods. All the output of the executed ROOT macros are then saved in the file [output](output.txt).

In [12]:
%%capture cap --no-stderr
#previouus line: capture std_output for future analysis 

# Train MVAs
factory.TrainAllMethods()

# Test MVAs
factory.TestAllMethods()

# Evaluate MVAs
factory.EvaluateAllMethods() 


In [13]:
#write on file standard output 
with open('std_output.txt', 'w') as f:
     f.write(cap.stdout)
# Save the output.
outputFile.Close()